In [16]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [31]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [32]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [33]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [34]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [35]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [36]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chathuka dilakshana\AppData\Local\Temp\ipykernel_13128\869911806.py:6: SyntaxWarning: invalid escape sequence '\d'
  data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)


In [37]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [38]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [39]:
txt = "awesome product. i love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'

In [40]:
txt

'awesome product. i love it'

In [42]:
txt1

'I love it this is amazing product'

In [44]:
txt1 ='I love it this is amazing product'
preprocessed_txt1 = preprocessing(txt1)
vectorized_txt1 = vectorizer(preprocessed_txt1, tokens)
prediction = get_prediction(vectorized_txt1)
prediction

'positive'

In [45]:
txt1

'I love it this is amazing product'

In [46]:
preprocessed_txt1

0    love amaz product
Name: tweet, dtype: object

In [47]:
txt2 ='I do not love it '
preprocessed_txt2 = preprocessing(txt2)
vectorized_txt2 = vectorizer(preprocessed_txt2, tokens)
prediction = get_prediction(vectorized_txt2)
prediction

'positive'

In [48]:
preprocessed_txt2

0    love
Name: tweet, dtype: object